In [13]:
import import_hack
import numpy as np
import core.steps as steps
from core import data_store as ds
from core import transformer as tr
from core.common.ds_utils import print_ds_items_info
from core.quantization.pq_quantizer import PQQuantizer, restore_from_clusters, build_pq_params_str
from core.common.path_helper import DataStoreHelper
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
ds_helper=DataStoreHelper(r'C:\data\computation\brodatz')

# SDC search requirement - compute centroids pairwise distances

In [15]:
def compute_centroids_pairwise_distances(descriptor_name, pq_params):
    centroids_ds = ds_helper.centroids_ds(descriptor_name, pq_params)
    centroids = ds.get_as_array(centroids_ds)
    centroids_pairwise_distances_ds = ds_helper.centroids_pairwise_distances_ds(descriptor_name, pq_params)
    steps.transform_step(centroids_ds, [tr.VectorsToPairwiseDistances()], centroids_pairwise_distances_ds)

In [16]:
K_arr = [2 ** i for i in [4,5,6,7,8]]
m_arr = [m for m in [1, 2, 4, 8, 16]]
pq_params_arr = [{'n_clusters': K, 'n_quantizers': m} for K in K_arr for m in m_arr]

descriptor_names=['histograms', 'glcms','lbphistograms']
choosen_bovwproductbincounts_pq_params= [{'n_clusters': K, 'n_quantizers': m} for K,m in [(64,4),(128,1),(16,1)]]
descriptor_names+=ds_helper.bovw_descriptors_names('bovwproductbincounts',choosen_bovwproductbincounts_pq_params)

In [17]:
for descriptor_name in descriptor_names:
    for pq_params in pq_params_arr:
        compute_centroids_pairwise_distances(descriptor_name, pq_params)